In [2]:
import pandas as pd 
import numpy as np
import os
import csv
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score


In [3]:
os.chdir('C:\\Users\\grace\\UNSW\\COMP4121\\COMP4121_MajorProject\\Data')

In [4]:
#Load datasets
raw_sales_df = pd.read_csv('raw_sales.csv', parse_dates=['datesold'])
ma_lga_df = pd.read_csv('ma_lga_12345.csv', parse_dates=['saledate'])

C:\Users\grace\AppData\Local\Temp\ipykernel_11700\3741783504.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ma_lga_df = pd.read_csv('ma_lga_12345.csv', parse_dates=['saledate'])


In [5]:
# Show the first few rows of each dataset to understand their structure
print("Raw Sales Data:")
print(raw_sales_df.head())
print("\nMA LGA Data:")
print(ma_lga_df.head())

Raw Sales Data:
    datesold  postcode   price propertyType  bedrooms
0 2007-02-07      2607  525000        house         4
1 2007-02-27      2906  290000        house         3
2 2007-03-07      2905  328000        house         3
3 2007-03-09      2905  380000        house         4
4 2007-03-21      2906  310000        house         3

MA LGA Data:
    saledate      MA   type  bedrooms
0 2007-09-30  441854  house         2
1 2007-12-31  441854  house         2
2 2008-03-31  441854  house         2
3 2008-06-30  441854  house         2
4 2008-09-30  451583  house         2


In [6]:
#Preprocessing 'raw_sales_df'

# Convert 'datesold' to datetime and sort by this column
raw_sales_df['datesold'] = pd.to_datetime(raw_sales_df['datesold'], errors='coerce')
raw_sales_df = raw_sales_df.sort_values('datesold')

# Handle missing values (if any)
raw_sales_df = raw_sales_df.dropna(subset=['datesold', 'price', 'postcode', 'propertyType', 'bedrooms'])

# Encode categorical variables like 'propertyType' (house/unit)
label_encoder = LabelEncoder()
raw_sales_df['propertyType'] = label_encoder.fit_transform(raw_sales_df['propertyType'])

# One-hot encode postcode as a new feature (optional)
#raw_sales_df = pd.get_dummies(raw_sales_df, columns=['postcode'], drop_first=True)


In [ ]:
#Preprocessing 'ma_lga_df' & cleaning 'ma_lga_12345' DataFrame

# Convert 'saledate' to datetime and sort by this column
ma_lga_df['saledate'] = pd.to_datetime(ma_lga_df['saledate'], errors='coerce')
ma_lga_df = ma_lga_df.sort_values('saledate')

# Handle missing values (if any)
ma_lga_df = ma_lga_df.dropna(subset=['saledate', 'MA', 'type', 'bedrooms'])

# Encode categorical variables like 'type' (house/unit)
ma_lga_df['type'] = label_encoder.fit_transform(ma_lga_df['type'])

In [8]:
#Data Exploration

#Print basic info to inspect column types and missing values
print("\nRaw Sales Data Info:")
print(raw_sales_df.info())

print("\nMA LGA Data Info:")
print(ma_lga_df.info())

# Show basic statistics on the numerical columns
print("\nRaw Sales Data Description:")
print(raw_sales_df.describe())

print("\nMA LGA Data Description:")
print(ma_lga_df.describe())


Raw Sales Data Info:
<class 'pandas.core.frame.DataFrame'>
Index: 29580 entries, 0 to 24551
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datesold      29580 non-null  datetime64[ns]
 1   postcode      29580 non-null  int64         
 2   price         29580 non-null  int64         
 3   propertyType  29580 non-null  int32         
 4   bedrooms      29580 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(3)
memory usage: 1.2 MB
None

MA LGA Data Info:
<class 'pandas.core.frame.DataFrame'>
Index: 347 entries, 49 to 346
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   saledate  347 non-null    datetime64[ns]
 1   MA        347 non-null    int64         
 2   type      347 non-null    int32         
 3   bedrooms  347 non-null    int64         
dtypes: datetime64[ns](1), int32(1), int64(2)
memory usage

In [13]:
print(raw_sales_df[['postcode', 'propertyType']].isnull().sum())

postcode        0
propertyType    0
dtype: int64


In [14]:
print(raw_sales_df.head())
print(raw_sales_df.dtypes)


    datesold  postcode     price  propertyType  bedrooms
0 2007-02-07      2607  525000.0             0         4
1 2007-02-27      2906  290000.0             0         3
2 2007-03-07      2905  328000.0             0         3
3 2007-03-09      2905  380000.0             0         4
4 2007-03-21      2906  310000.0             0         3
datesold        datetime64[ns]
postcode                 int64
price                  float64
propertyType             int32
bedrooms                 int32
dtype: object


In [9]:
# ---------------------- Data Transformation for HMM ----------------------
# For the HMM, we need to create sequences of data that reflect transitions over time.

# Raw Sales: Create sequences based on 'datesold'
# Sort by date and group by 'postcode' (or other features)
raw_sales_df['price'] = raw_sales_df['price'].astype(float)  # Ensure price is float for HMM
raw_sales_df['bedrooms'] = raw_sales_df['bedrooms'].astype(int)  # Ensure bedrooms are int

# For simplicity, let's create sequences for one postcode and property type (you can generalize this)
# Creating sequences for HMM (e.g., price and bedrooms)
sequence_raw_sales = raw_sales_df[['datesold', 'price', 'bedrooms']].groupby(['postcode', 'propertyType']).apply(
    lambda group: group[['price', 'bedrooms']].values.tolist()).reset_index(name='sequence')

print("\nSequences for Raw Sales (Price and Bedrooms):")
print(sequence_raw_sales.head())

# MA LGA: Create sequences based on 'saledate'
# Similar process for 'ma_lga_12345.csv'
ma_lga_df['MA'] = ma_lga_df['MA'].astype(float)  # Ensure 'MA' is float for HMM
ma_lga_df['bedrooms'] = ma_lga_df['bedrooms'].astype(int)  # Ensure bedrooms are int

# Create sequences for MA and bedrooms based on saledate
sequence_ma_lga = ma_lga_df[['saledate', 'MA', 'bedrooms']].groupby(['type']).apply(
    lambda group: group[['MA', 'bedrooms']].values.tolist()).reset_index(name='sequence')

print("\nSequences for MA LGA (Moving Average and Bedrooms):")
print(sequence_ma_lga.head())

# ---------------------- Saving Preprocessed Data ----------------------
# Save processed data to new CSVs for future use
raw_sales_df.to_csv('processed_raw_sales.csv', index=False)
ma_lga_df.to_csv('processed_ma_lga_12345.csv', index=False)

# Save sequences for HMM model
sequence_raw_sales.to_csv('sequence_raw_sales.csv', index=False)
sequence_ma_lga.to_csv('sequence_ma_lga.csv', index=False)

print("\nData Preprocessing Complete!")

KeyError: 'postcode'